К вам на анализ попал набор данных о работе сервиса по продаже авиабилетов с дневной агрегацией. Вашей задаче будет рассчитать определенные показатели для каждого месяца, представленные ниже.

Данные содержат:

Дату

Идентификатор сессии

Идентификатор пользователя

Сумму оплаты во время сессии

Оценку сервиса (она запрашивается у пользователя после оформления билетов)
----------------------------------------
 

Используя данные, рассчитайте показатели для каждого месяца:

Количество сессий

MAU

WAU (среднее)

DAU (среднее)

Липкость 

Конверсию в продажу

ARPU

ARPPU

Удовлетворенность клиентов

Для второй недели января рассчитайте Retention с 1 по 10 недели. Постройте график, сделайте выводы.

Загрузите ваш notebook на или укажите ссылку на облачное хранилище (не забудьте открыть доступ к документу).


In [1]:
#импорты
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import copy

In [2]:
#подключение таблиц
df = pd.read_csv(r'.\data.csv')
df

,Дата,ID сессии,ID клиента,Доход,Оценка
0,2021-01-05,eeehjltlceclayhacecerhyyxhrxs,45870310,0.0,NaN
1,2021-01-06,eeehrcecacecerlrhacecerygejeb,76545402,0.0,NaN
2,2021-01-06,eeehalhalhjaracecerxhrhtrm,424242,0.0,NaN
3,2021-01-06,eeehrcecacecerlrhacecerygejeb,76545402,0.0,NaN
4,2021-01-06,eeehalhalhjaracecerxhrhtrm,424242,0.0,NaN
...,...,...,...,...,...
19005,2021-03-30,aleyeececegrjjacecerrnmslehl,25114341,0.0,NaN
19006,2021-03-30,aleyeececegrjjacecerrnmslehl,25114341,0.0,NaN
19007,2021-03-30,aleyeececegrjjacecerrnmslehl,25114341,0.0,NaN
19008,2021-03-30,aleyeheealhejacecerkprcecehf,12244122,0.0,NaN


In [83]:
# количество ссеий по дням
user_session_count = df.groupby(['Дата']).agg(
    {
        'ID сессии': 'count',
        'Доход': 'sum',
        'Оценка': 'mean'
    }
).reset_index()
user_session_count = user_session_count.rename(columns={'ID сессии': 'Количество сессий'})
user_session_count.sort_values(by='Доход', ascending= False)

,Дата,Количество сессий,Доход,Оценка
30,2021-02-05,673,7.712588e+06,4.021739
29,2021-02-04,497,6.143709e+06,3.957447
31,2021-02-06,536,5.573908e+06,3.974545
62,2021-03-09,501,5.332308e+06,4.022140
28,2021-02-03,337,3.654363e+06,3.964497
...,...,...,...,...
5,2021-01-10,2,0.000000e+00,NaN
4,2021-01-09,1,0.000000e+00,NaN
3,2021-01-08,2,0.000000e+00,NaN
2,2021-01-07,2,0.000000e+00,NaN


In [75]:
# количество входов в день
daily_client_count = df.groupby(['Дата'])['ID клиента'].agg(['count']).reset_index()
daily_client_count = daily_client_count.rename(columns={'count': 'Количество посещений в день'})
daily_client_count

,Дата,Количество посещений в день
0,2021-01-05,1
1,2021-01-06,4
2,2021-01-07,2
3,2021-01-08,2
4,2021-01-09,1
...,...,...
77,2021-03-25,5
78,2021-03-26,6
79,2021-03-27,2
80,2021-03-29,1


In [5]:
# Копируем таблицу, переводим дату и достаем месяц

df_month_copy = df.copy()
df_month_copy['Дата'] = pd.to_datetime(df_month_copy['Дата'])
df_month_copy['Дата'] = df_month_copy['Дата'].dt.to_period('M').dt.strftime('%Y.%m')

# количество входов в месяц
monthly_client_count = df_month_copy.groupby('Дата')['ID клиента'].agg(['count']).reset_index()
monthly_client_count = monthly_client_count.rename(columns={'count': 'Количество посещений в месяц'})

monthly_client_count

,Дата,Количество посещений в месяц
0,2021.01,3304
1,2021.02,10740
2,2021.03,4966


In [6]:
# количество входов в месяц
monthly_client_MAU = df_month_copy.groupby('Дата')['ID клиента'].nunique().reset_index()
monthly_client_MAU = monthly_client_MAU.rename(columns={'ID клиента': 'Количество уникальных посещений'})

monthly_client_MAU


,Дата,Количество уникальных посещений
0,2021.01,2495
1,2021.02,7546
2,2021.03,3610


In [60]:
#расчет WAU(среднее)
df_weekly_copy = df.copy()
df_weekly_copy['Дата'] = pd.to_datetime(df_weekly_copy['Дата'])
df_weekly_copy['Дата'] = df_weekly_copy['Дата'].dt.to_period('W').dt.strftime('%W')
df_weekly_copy = df_weekly_copy.rename(columns={'Дата': 'Неделя'})

# количество входов в неделю
df_weekly_WAU = df_weekly_copy.groupby('Неделя')['ID клиента'].nunique().reset_index()
df_weekly_WAU = df_weekly_WAU.rename(columns={'ID клиента': 'Количество уникальных посещений'})

df_weekly_WAU




,Неделя,Количество уникальных посещений
0,01,8
1,02,741
2,03,1010
3,04,829
4,05,2146
5,06,2026
6,07,2031
7,08,1871
8,09,2269
9,10,1413


In [61]:
# понедельный доход
df_weekly_copy.groupby('Неделя')['Доход'].agg(['sum']).astype('int64').reset_index()

,Неделя,sum
0,01,0
1,02,5344878
2,03,11235411
3,04,10594940
4,05,28472954
5,06,87884
6,07,241478
7,08,215007
8,09,2181525
9,10,16969445
